## Importing all the libraries

In [18]:
import pandas as pd
import requests
import numpy as np
import folium
from sklearn.cluster import KMeans
from geopy.geocoders import Nominatim
from bs4 import BeautifulSoup

# #Getting table from Wikipedia

In [19]:
website_url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
from bs4 import BeautifulSoup
soup = BeautifulSoup(website_url,'lxml')
#print(soup.prettify())

My_table = soup.find('table',{'class':'wikitable sortable'})
links = My_table.findAll('tr')[1:]
Postcode = []
Borough = []
Neighborhood = []
for link in links:
    x = link.find_all('td')[0]
    y = link.find_all('td')[2]
    Postcode.append(x.text)
    Borough.append(y.text)
    z = link.find_all('td')[1]
    Neighborhood.append(z.text)

## Creating a data frame from the table

In [20]:
df = pd.DataFrame(columns=['Postcode','Borough','Neighborhood'])

## Filling the pandas data frame

In [21]:
df['Postcode'] = Postcode
df['Borough'] = Neighborhood
df['Neighborhood'] = Borough

## cleaning the neighborhood coloumn

In [22]:
x = df['Neighborhood']
y = list(x)
z = [col.split('\n')[0] for col in y]
df['Neighborhood'] = z

## Removing all NaN values

In [23]:
df.replace('Not assigned','NaN', inplace = True)
borough_not_required = df[df['Borough']== 'NaN'].index.tolist()
df = df.drop(borough_not_required).reset_index(drop=True)
df.head()

,Postcode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights


## Joining the Neighborhood coloumn for the same Postcode

In [24]:
x  = df.groupby(['Postcode','Borough'])['Neighborhood'].apply(','.join)
toronto = pd.DataFrame(x) 
toronto.reset_index(inplace=True)
toronto.head()

,Postcode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [25]:
toronto.shape

(103, 3)